In [1]:
import pandas as pd 
import numpy as np 
import requests as rqs 
from bs4 import BeautifulSoup as BS 

In [2]:
categories = []
names = []
prices = []
plinks = []

In [3]:
source = rqs.get("https://www.jumia.co.ke/").text 
soup = BS(source, "lxml") 

In [4]:
main = soup.find("main", class_="has-b2top")
holder = main.find("div", class_="flyout")
links = holder.find_all("a", class_="itm") 

In [5]:
for link in links:
    try:
        categories.append(link["href"])
    except KeyError:
        pass

In [6]:
categories.pop(0)

'https://www.jumia.co.ke/groceries/'

In [7]:
for category in categories:
    for x in range(1, 51):
        link = f"https://www.jumia.co.ke/{category}/?page={x}"
        source = rqs.get(link).text
        soup = BS(source, "lxml")
        for article in soup.find_all("article", class_="prd _fb col c-prd"):
            trail = article.find("a", class_="core")["href"] 
            link = f"https://www.jumia.co.ke/{category}{trail}"  
            name = article.find("div", class_="info").find("h3", class_="name").text
            price = article.find("div", class_="info").find("div", class_="prc").text
            names.append(name)
            prices.append(price)
            plinks.append(link) 

In [8]:
names = pd.Series(names)
prices = pd.Series(prices) 
plinks = pd.Series(plinks)

In [9]:
df = pd.DataFrame() 

In [10]:
df.insert(0, "Name", names) 
df.insert(1, "Price", prices) 
df.insert(2, "Link", plinks) 

In [11]:
df

,Name,Price,Link
0,Kiss Beauty Bb Bb Even Skin Tone Beautifying F...,KSh 675,https://www.jumia.co.ke//health-beauty//kiss-b...
1,Carrot Glow Toning Lightening Serum,KSh 405,https://www.jumia.co.ke//health-beauty//carrot...
2,Lifesum Methylated Spirit 70% Vv 5 Litres,"KSh 1,221",https://www.jumia.co.ke//health-beauty//lifesu...
3,Garnier Pure Active 3 In1 Charcoal Anti Blackh...,KSh 699,https://www.jumia.co.ke//health-beauty//garnie...
4,Dr. Rashel Vitamin C Brightening & Anti-Aging ...,KSh 322,https://www.jumia.co.ke//health-beauty//dr.-ra...
...,...,...,...
19995,Bosch GSA 18 V-LI Cordless Sabre Saw,"KSh 34,210",https://www.jumia.co.ke//patio-lawn-garden//bo...
19996,Bosch Palm Router GKF 600 - Blue & black,"KSh 28,420",https://www.jumia.co.ke//patio-lawn-garden//bo...
19997,Generic 8-32mm Steel Silver Metric Spanner Wre...,"KSh 1,759",https://www.jumia.co.ke//patio-lawn-garden//ge...
19998,AICO Heavy Duty Vacuum Cleaner Wet & Dry 50L -...,"KSh 24,000",https://www.jumia.co.ke//patio-lawn-garden//he...


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    20000 non-null  object
 1   Price   20000 non-null  object
 2   Link    20000 non-null  object
dtypes: object(3)
memory usage: 468.9+ KB


In [13]:
def replacer(price):
    price = price.replace("KSh", "")
    price = price.replace(",", "")
    return price

In [14]:
df["Price"] = df["Price"].apply(replacer) 

In [15]:
def check(price):
    if "-" in price:
        price = price.replace("-", "").split(" ")[1]
    return price

In [16]:
df["Price"] = df["Price"].apply(check) 

In [17]:
df["Price"] = pd.to_numeric(df["Price"]) 

In [18]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    20000 non-null  object
 1   Price   20000 non-null  int64 
 2   Link    20000 non-null  object
dtypes: int64(1), object(2)
memory usage: 468.9+ KB


In [19]:
df = df.sort_values(["Price"], ascending=False).reset_index(drop=True) 

In [20]:
df.head(10) 

,Name,Price,Link
0,Sony KD - 65A9G 65 Inch TV - MASTER Series - ...,780000,https://www.jumia.co.ke//electronics//sony-kd-...
1,Generic Tumo Int 5000W 3 Blades Wind Turbine G...,599880,https://www.jumia.co.ke//patio-lawn-garden//ge...
2,DELL XPS 13 7390 CORE I7 10510U 16GB 512SSD WI...,244230,https://www.jumia.co.ke//computing//dell-xps-1...
3,Bosch Magnetic Drill Stand GMB 32 Professional,236250,https://www.jumia.co.ke//patio-lawn-garden//bo...
4,Generic SnoMaster 92.5L Dual Compartment Stain...,215710,https://www.jumia.co.ke//sporting-goods//gener...
5,"Sony COMPUTER ENTERTAINMENT PS5 CONSOLE,825GB,...",190000,https://www.jumia.co.ke//video-games//sony-com...
6,"Sony COMPUTER ENTERTAINMENT PS5 CONSOLE,825GB,...",190000,https://www.jumia.co.ke//video-games//sony-com...
7,"Apple 12.9"" IPad Pro (Early 2020, 128GB, Wi-Fi...",185000,https://www.jumia.co.ke//phones-tablets//apple...
8,Solar Africa Factory Sale Off Grid Solar Power...,180999,https://www.jumia.co.ke//patio-lawn-garden//fa...
9,Generic Husqvarna Power Saw,180000,https://www.jumia.co.ke//patio-lawn-garden//hu...
